In [1]:
## Import Ticker class from yahooquery library 
# To access data from yahooquery, an argument (Company ticker symbol) is passed to the Ticker class 

from yahooquery import Ticker
import pandas as pd

In [2]:
## Create instance of Ticker class (Company ticker symbol)

WES_ASX = Ticker(['WES.AX'])

In [ ]:
## Market Value Ratios 

# Market value ratios are used to evaluate the current share price of a company 
# Typically the objective of market value ratios is to identify undervalued or overvalued companies

# The ratio formulas comprise of data from the company's financial statements, as well as technical elements from the stock performance

In [ ]:
# Select the Income Statement from the Ticker class, this will return a DataFrame

incomeStatement = WES_ASX.income_statement().reset_index()

In [10]:
# Select History from Ticker class, this will return a DataFrame of historical stock price data 
# A period of 5 years has been selected for historical analysis 

stockPrice = WES_ASX.history(period="5y")

# Print results 

stockPrice

open       high        low      close   volume  \
symbol date                                                              
WES.AX 2018-04-03  29.534424  29.666082  29.363623  29.498838  2100812   
       2018-04-04  29.427671  29.559332  29.249754  29.470373  2397371   
       2018-04-05  29.399204  29.847559  29.399204  29.712341  2010585   
       2018-04-06  29.555773  29.744368  29.392090  29.605591  1720737   
       2018-04-09  29.634056  29.918726  29.612707  29.840443  1575555   
...                      ...        ...        ...        ...      ...   
       2023-03-28  49.930000  50.400002  49.380001  49.580002  1385052   
       2023-03-29  49.330002  50.029999  49.119999  50.029999  1528545   
       2023-03-30  50.250000  50.380001  49.660000  49.759998  1683754   
       2023-03-31  50.000000  50.459999  49.910000  50.250000  1580209   
       2023-04-03  50.500000  51.230000  50.500000  51.180000  1458721   

                    adjclose  dividends  
symbol date                              
WES.AX 2018-04-03  23.630686        0.0  
       2018-04-04  23.607883        0.0  
       2018-04-05  23.801716        0.0  
       2018-04-06  23.716200        0.0  
       2018-04-09  23.904335        0.0  
...                      ...        ...  
       2023-03-28  49.580002        0.0  
       2023-03-29  50.029999        0.0  
       2023-03-30  49.759998        0.0  
       2023-03-31  50.250000        0.0  
       2023-04-03  51.180000        0.0  

[1268 rows x 7 columns]

In [ ]:
# Price to Earnings Ratio (P/E)